---
title: Utils for Polars
---

In [ ]:
#| default_exp utils/polars

In [ ]:

#| export
import polars as pl
import modin.pandas as mpd

from typing import Any, Collection

from functools import partial


In [ ]:
#| export
def create_partitions(files, func):
    keys = [file.split("/")[-1] for file in files]
    return {key: partial(func, file) for key, file in zip(keys, files)}

## IO

In [ ]:
#| export
def convert_to_pd_dataframe(
    df: pl.DataFrame | pl.LazyFrame, # original DataFrame or LazyFrame
    modin: bool = False # whether to use modin or not
):
    """
    Convert a Polars DataFrame or LazyFrame into a pandas-like DataFrame.
    If modin=True, returns a Modin DataFrame.
    """
    if isinstance(df, pl.LazyFrame):
        df = df.collect()
    elif not isinstance(df, pl.DataFrame):
        raise TypeError("Input must be a Polars DataFrame or LazyFrame")

    data = df.to_pandas(use_pyarrow_extension_array=True)

    if modin:
        return mpd.DataFrame(data)
    else:
        return data


## Functions

In [ ]:
#| export
def sort(df: pl.DataFrame, col="time"):
    if df.get_column(col).is_sorted():
        return df.set_sorted(col)
    else:
        return df.sort(col)


In [ ]:
#| export
def _expand_selectors(items: Any, *more_items: Any) -> list[Any]:
    """
    See `_expand_selectors` in `polars`.
    """
    expanded: list[Any] = []
    for item in (
        *(
            items
            if isinstance(items, Collection) and not isinstance(items, str)
            else [items]
        ),
        *more_items,
    ):
        expanded.append(item)
    return expanded

In [ ]:
#| export

def pl_norm(columns, *more_columns) -> pl.Expr:
    """
    Computes the square root of the sum of squares for the given columns.

    Args:
    *columns (str): Names of the columns.

    Returns:
    pl.Expr: Expression representing the square root of the sum of squares.
    """
    all_columns = _expand_selectors(columns, *more_columns)
    squares = [pl.col(column).pow(2) for column in all_columns]

    return sum(squares).sqrt()

In [ ]:
# | export
def decompose_vector(
    df: pl.DataFrame, vector_col, name=None, suffixes: list = ["_x", "_y", "_z"]
):
    """
    Decompose a vector column in a DataFrame into separate columns for each component with custom suffixes.

    Parameters:
    - df (pl.DataFrame): The input DataFrame.
    - vector_col (str): The name of the vector column to decompose.
    - name (str, optional): Base name for the decomposed columns. If None, uses `vector_col` as the base name.
    - suffixes (list, optional): A list of suffixes to use for the decomposed columns.
      If None or not enough suffixes are provided, defaults to '_0', '_1', etc.

    Returns:
    - pl.DataFrame: A DataFrame with the original vector column decomposed into separate columns.
    """

    if name is None:
        name = vector_col

    # Determine the maximum length of vectors in the column to handle dynamic vector lengths
    max_length = df.select(pl.col(vector_col).list.len()).max()[0, 0]

    if suffixes is None or len(suffixes) < max_length:
        if suffixes is None:
            suffixes = []
        # Extend or create the list of suffixes with default values
        suffixes.extend([f"_{i}" for i in range(len(suffixes), max_length)])

    # Create column expressions for each element in the vector
    column_expressions = [
        pl.col(vector_col).list.get(i).alias(name).name.suffix(suffixes[i])
        for i in range(max_length)
    ]

    return df.with_columns(column_expressions)